# Local Real-Time Inference

Complete workflow for serving MLflow models locally.

**Steps:**
1. Load config & discover latest run
2. Load signature / feature columns artifact
3. Prepare & align sample data
4. Serve model locally (separate terminal)
5. Health check & prediction via HTTP

In [ ]:
import os, json, time, logging
import pandas as pd
import mlflow
import yaml
import requests
from mlflow.tracking import MlflowClient

import sys
sys.path.append(os.path.abspath('../'))

from utils.common_utils import load_config, setup_logging
from utils.data_loader import load_data_from_source

config = load_config('../config/config.yaml')
setup_logging(config)

# Ensure tracking URI is set BEFORE creating a client
mode = config.get('environment', {}).get('mode', 'local')
if mode == 'databricks':
    tracking_uri = config['mlflow']['databricks']['tracking_uri']
else:
    tracking_uri = config['mlflow']['local']['tracking_uri']
mlflow.set_tracking_uri(tracking_uri)
print(f"Tracking URI set to: {tracking_uri}")

client = MlflowClient()
print("MlflowClient initialized.")

## 1. Discover Latest Run

In [ ]:
def get_latest_run_from_filesystem(mlruns_path='../mlruns', experiment_name=None):
    """Fallback: Scan filesystem directly when MLflow client can't see experiments"""
    import os
    import yaml
    from pathlib import Path
    
    mlruns = Path(mlruns_path)
    if not mlruns.exists():
        return None
    
    latest_run = None
    latest_time = 0
    
    # Scan all experiment directories
    for exp_dir in mlruns.iterdir():
        if not exp_dir.is_dir() or exp_dir.name in ['.trash', 'models']:
            continue
        
        # Check experiment meta
        meta_file = exp_dir / 'meta.yaml'
        if not meta_file.exists():
            continue
        
        try:
            with open(meta_file, 'r') as f:
                exp_meta = yaml.safe_load(f)
            
            # Filter by experiment name if specified
            if experiment_name and exp_meta.get('name') != experiment_name:
                continue
            
            # Scan runs in this experiment
            for run_dir in exp_dir.iterdir():
                if not run_dir.is_dir() or run_dir.name in ['models', 'tags']:
                    continue
                
                run_meta_file = run_dir / 'meta.yaml'
                if not run_meta_file.exists():
                    continue
                
                with open(run_meta_file, 'r') as f:
                    run_meta = yaml.safe_load(f)
                
                start_time = run_meta.get('start_time', 0)
                if start_time > latest_time:
                    latest_time = start_time
                    latest_run = {
                        'run_id': run_meta['run_id'],
                        'experiment_id': run_meta['experiment_id'],
                        'start_time': start_time,
                        'status': run_meta.get('status')
                    }
        except Exception as e:
            logging.debug(f"Error scanning {exp_dir}: {e}")
            continue
    
    return latest_run

mode = config.get('environment', {}).get('mode', 'local')
if mode == 'databricks':
    experiment_name = config['mlflow']['databricks']['experiment_name']
else:
    experiment_name = config['mlflow']['local']['experiment_name']
print(f"Environment mode: {mode}")
print(f"Target experiment name: {experiment_name}")

# Try filesystem scan as fallback
latest_run_info = get_latest_run_from_filesystem('../mlruns', experiment_name)
if not latest_run_info:
    raise RuntimeError(
        f"No runs found for experiment '{experiment_name}'. Retrain the model first."
    )

run_id = latest_run_info['run_id']
print(f'✅ Latest run: {run_id}')
print(f'   Experiment ID: {latest_run_info["experiment_id"]}')

## 2. Load Signature & Feature Columns

In [ ]:
def load_signature_and_features(run_id, experiment_id, mlruns_path='../mlruns'):
    """Load signature and features directly from filesystem"""
    from pathlib import Path
    
    signature_cols = None
    feature_cols = None
    
    # Build artifact path
    run_dir = Path(mlruns_path) / experiment_id / run_id / 'artifacts'
    
    # Load signature from MLmodel
    try:
        mlmodel_path = run_dir / 'model' / 'MLmodel'
        if mlmodel_path.exists():
            with open(mlmodel_path, 'r', encoding='utf-8') as f:
                yml = yaml.safe_load(f)
            sig_inputs = yml.get('signature', {}).get('inputs', [])
            signature_cols = [i.get('name') for i in sig_inputs if i.get('name')]
            if signature_cols:
                print(f'✅ Loaded signature: {len(signature_cols)} columns')
        else:
            print(f'⚠️  MLmodel not found at {mlmodel_path}')
    except Exception as e:
        logging.warning(f'Signature load failed: {e}')
    
    # Load feature columns artifact
    try:
        feat_art = run_dir / 'feature_columns.json'
        if feat_art.exists():
            with open(feat_art, 'r') as f:
                data = json.load(f)
            feature_cols = data.get('feature_columns')
            if feature_cols:
                print(f'✅ Loaded feature artifact: {len(feature_cols)} columns')
        else:
            print(f'⚠️  feature_columns.json not found at {feat_art}')
    except Exception as e:
        logging.warning(f'Feature load failed: {e}')
    
    return signature_cols, feature_cols

signature_cols, feature_cols = load_signature_and_features(run_id, latest_run_info['experiment_id'])
print(f'Signature: {len(signature_cols) if signature_cols else None}')
print(f'Feature artifact: {len(feature_cols) if feature_cols else None}')

## 3. Prepare Sample Data

In [ ]:
if config['data_source']['type'] == 'unity_catalog':
    from utils.common_utils import get_spark_session
    spark = get_spark_session(config)
    features_df = load_data_from_source(config, 'customer_features', spark)
else:
    features_df = pd.read_csv('../data/processed/customer_features.csv')

sample = features_df.sample(1, random_state=42)
customer_id = sample['CUSTOMERID'].iloc[0]
X_raw = sample.drop(['CUSTOMERID','PARTYID'], axis=1, errors='ignore')

cat_cols = X_raw.select_dtypes(include=['object','category']).columns.tolist()
if cat_cols:
    X_raw = pd.get_dummies(X_raw, columns=cat_cols, drop_first=True)

print(f'Customer ID: {customer_id}')
print(f'Raw columns: {len(X_raw.columns)}')

## 4. Align Columns

In [ ]:
expected = signature_cols or feature_cols
assert expected, 'No schema available. Retrain model after patch.'

def align(df, expected_cols):
    for c in expected_cols:
        if c not in df.columns:
            df[c] = 0
    to_drop = [c for c in df.columns if c not in expected_cols]
    if to_drop:
        df = df.drop(columns=to_drop)
    return df[expected_cols]

X_aligned = align(X_raw.copy(), expected)
print(f'Aligned shape: {X_aligned.shape}')
missing = [c for c in expected if c not in X_aligned.columns]
extra = [c for c in X_aligned.columns if c not in expected]
print(f'Missing: {len(missing)} | Extra: {len(extra)}')

## 5. Serve Commands

Copy and run in a **separate PowerShell terminal**:

In [ ]:
print(f'Run-based (recommended):')
print(f'mlflow models serve -m "runs:/{run_id}/model" -p 5000 --env-manager=local')
print()
model_name = config['mlflow']['local']['registered_model_name']
print(f'Registry-based:')
print(f'mlflow models serve -m "models:/{model_name}/Production" -p 5000 --env-manager=local')

## 6. Health Check & Prediction

In [ ]:
ENDPOINT = 'http://127.0.0.1:5000'

def check_health(url=ENDPOINT):
    try:
        r = requests.get(url + '/health', timeout=5)
        return r.status_code == 200
    except Exception:
        return False

healthy = check_health()
print(f'Health: {healthy}')

if healthy:
    payload = {'dataframe_split': {'columns': X_aligned.columns.tolist(), 'data': X_aligned.values.tolist()}}
    r = requests.post(ENDPOINT + '/invocations', json=payload, timeout=30)
    print(f'Status: {r.status_code}')
    if r.status_code == 200:
        result = r.json()
        print(json.dumps(result, indent=2))
        if 'predictions' in result:
            print(f'Recommended Product: {result["predictions"][0]}')
    else:
        print(r.text[:500])
else:
    print('Start server first')